In [ ]:
import sys
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cmocean as cmo
import cartopy.crs as ccrs

sys.path.append("..")

from plotfunctions import *

np.seterr(all='ignore')
import warnings
warnings.filterwarnings("ignore")

%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
mpl.rcParams['figure.subplot.bottom'] = .15
mpl.rcParams['figure.subplot.top']    = .94
mpl.rcParams['figure.subplot.left']   = .01
mpl.rcParams['figure.subplot.right']  = .99
mpl.rcParams['figure.subplot.wspace'] = .02
mpl.rcParams['figure.subplot.hspace'] = .0

In [ ]:
"""Locations for text"""
plon,plat = {},{}
plon['CrossDots'] = 250.5
plat['CrossDots'] = -74.4
plon['Thwaites_e'] = 253.5
plat['Thwaites_e'] = -74.5
plon['PineIsland'] = 257
plat['PineIsland'] = -74.8

In [ ]:
def printvals(ax,geom,dmelt):
    ax.text(plon[geom],plat[geom],f'+{np.nanmean(dmelt):.1f}',transform=ccrs.PlateCarree(),c='w',ha='center')

In [ ]:
#Results date
#date = '2021_02_03/'
date = ''

#Extension for filename
#ext = ''
ext = '_lowvis'

In [ ]:
fig = plt.figure(figsize=(8,7))
proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=-13)

ax1 = fig.add_subplot(141,projection=proj)
makebackground(ax1)
ax1.set_title('a) Plume',loc='left')
ax2 = fig.add_subplot(142,projection=proj)
makebackground(ax2)
ax2.set_title('b) PICO',loc='left')
ax3 = fig.add_subplot(143,projection=proj)
makebackground(ax3)
ax3.set_title('c) PICOP',loc='left')
ax4 = fig.add_subplot(144,projection=proj)
makebackground(ax4)
ax4.set_title('d) Layer',loc='left')

"""Layer"""

for geom in ['Thwaites_e','PineIsland','CrossDots']:
    ds = xr.open_dataset(f'../../results/{date}Layer_{geom}_tanh_Tdeep1.5_ztcl-550.nc')   
    melt1 = ds.melt
    ds = xr.open_dataset(f'../../results/{date}Layer_{geom}_tanh_Tdeep0.5_ztcl-550.nc')  
    ds = add_lonlat(ds)
    melt2 = ds.melt
    dmelt = np.where(ds.mask==3,melt1-melt2,np.nan)    
    printvals(ax4,geom,dmelt)
    IM = plotdiffmelt(ax4,ds.lon,ds.lat,dmelt)

"""Colorbar"""
ax = fig.add_subplot(141)
ax.set_visible(False)
axins = inset_axes(ax,width="400%",height="4%",loc='lower left',bbox_to_anchor=(0.,-.08, 1, 1),bbox_transform=ax.transAxes,borderpad=0)
cbar = plt.colorbar(IM, cax=axins,extend='max',orientation='horizontal')

cbar.set_label('Melt rate increase [m/yr]', labelpad=0)

"""Save figure"""
plt.savefig(f"../../figures/Warming{ext}.png",dpi=300)
plt.show()